In [1]:
import mlrose_hiive as mlrose
import numpy as np
import matplotlib.pyplot as plt
import time

# Define a function to create a TSP problem
def create_tsp_problem(num_cities):
    coords = [(np.random.rand(), np.random.rand()) for _ in range(num_cities)]
    fitness_dists = mlrose.TravellingSales(coords=coords)
    problem = mlrose.TSPOpt(length=num_cities, fitness_fn=fitness_dists, maximize=False)
    return problem

# Function to run algorithms and collect data
def run_experiment(problem, algorithm, **kwargs):
    start_time = time.time()
    best_state, best_fitness, curve = algorithm(problem, curve=True, **kwargs)
    end_time = time.time()
    duration = end_time - start_time
    fitness_curve = -1 * np.array(curve)  # Apply -1 to fitness
    return best_fitness, fitness_curve, duration

# Function to generate charts
def generate_charts(results, problem_sizes, algorithm, param_name, param_values):
    plt.figure(figsize=(14, 10))
    
    # Fitness / Iteration
    plt.subplot(2, 2, 1)
    for result in results:
        plt.plot(result['fitness_curve'], label=f'{param_name}={result["param_value"]}')
    plt.xlabel('Iteration')
    plt.ylabel('Fitness')
    plt.title(f'Fitness / Iteration ({algorithm})')
    plt.legend()

    # Fitness / Problem Size
    plt.subplot(2, 2, 2)
    avg_fitness = [np.mean([result['best_fitness'] for result in results if result['problem_size'] == size]) for size in problem_sizes]
    plt.plot(problem_sizes, avg_fitness, marker='o')
    plt.xlabel('Problem Size')
    plt.ylabel('Best Fitness')
    plt.title(f'Fitness / Problem Size ({algorithm})')

    # Function Evaluations and Wall Clock Time
    plt.subplot(2, 2, 3)
    evaluations = [result['duration'] for result in results]
    plt.plot(param_values, evaluations, marker='o')
    plt.xlabel(param_name)
    plt.ylabel('Wall Clock Time (s)')
    plt.title(f'Function Evaluations and Wall Clock Time ({algorithm})')

    # Best Fitness with Different Parameter Values
    plt.subplot(2, 2, 4)
    best_fitness = [result['best_fitness'] for result in results]
    plt.plot(param_values, best_fitness, marker='o')
    plt.xlabel(param_name)
    plt.ylabel('Best Fitness')
    plt.title(f'Best Fitness with Different {param_name} ({algorithm})')

    plt.tight_layout()
    plt.show()

# Experiment configurations
problem_sizes = [10, 20, 30, 40, 50]
sa_temperatures = [0.1, 1.0, 10.0, 100.0, 1000.0]
ga_cross_over_rates = [0.1, 0.3, 0.5, 0.7, 0.9]
results_sa = []
results_ga = []

# Run SA experiments
for temp in sa_temperatures:
    for size in problem_sizes:
        problem = create_tsp_problem(size)
        best_fitness, fitness_curve, duration = run_experiment(problem, mlrose.simulated_annealing, schedule=mlrose.GeomDecay(init_temp=temp), max_iters=1000)
        results_sa.append({'param_value': temp, 'problem_size': size, 'best_fitness': best_fitness, 'fitness_curve': fitness_curve, 'duration': duration})

# Run GA experiments
for rate in ga_cross_over_rates:
    for size in problem_sizes:
        problem = create_tsp_problem(size)
        best_fitness, fitness_curve, duration = run_experiment(problem, mlrose.genetic_alg, pop_size=200, mutation_prob=rate, max_iters=1000)
        results_ga.append({'param_value': rate, 'problem_size': size, 'best_fitness': best_fitness, 'fitness_curve': fitness_curve, 'duration': duration})

# Generate charts
generate_charts(results_sa, problem_sizes, 'SA', 'Temperature', sa_temperatures)
generate_charts(results_ga, problem_sizes, 'GA', 'Crossover Rate', ga_cross_over_rates)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()